In [3]:
import boto3
import s3fs
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

### Load in the datasets
fy14 = dd.read_parquet('s3://dask/2014_parking_violations/', dtype=dtypes, usecols=common_columns)
fy15 = dd.read_parquet('s3://dask/2015_parking_violations/', dtype=dtypes, usecols=common_columns)
fy16 = dd.read_parquet('s3://dask/2016_parking_violations/', dtype=dtypes, usecols=common_columns)
fy17 = dd.read_parquet('s3://dask/2017_parking_violations/', dtype=dtypes, usecols=common_columns)
fy17

,Summons_Number,Plate_ID,Registration_State,Plate_Type,Issue_Date,Violation_Code,Vehicle_Body_Type,Vehicle_Make,Issuing_Agency,Street_Code1,Street_Code2,Street_Code3,Vehicle_Expiration_Date,Violation_Location,Violation_Precinct,Issuer_Precinct,Issuer_Code,Issuer_Command,Issuer_Squad,Violation_Time,Time_First_Observed,Violation_County,Violation_In_Front_Of_Or_Opposite,House_Number,Street_Name,Intersecting_Street,Date_First_Observed,Law_Section,Sub_Division,Violation_Legal_Code,Days_Parking_In_Effect_,From_Hours_In_Effect,To_Hours_In_Effect,Vehicle_Color,Unregistered_Vehicle?,Vehicle_Year,Meter_Number,Feet_From_Curb,Violation_Post_Code,Violation_Description,No_Standing_or_Stopping_Violation,Hydrant_Violation,Double_Parking_Violation
npartitions=16,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,int32,object,object,object,int32,int32,int32,int32,float64,int32,int32,int32,object,object,object,object,object,object,object,object,object,int32,int32,object,object,object,object,object,object,float64,int32,object,int32,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [7]:
### Notice that the columns from each dataset are slightly different
print("fy14: " + fy14.columns)
print("fy17: " + fy17.columns)

Index(['fy14: Summons_Number', 'fy14: Plate_ID', 'fy14: Registration_State',
       'fy14: Plate_Type', 'fy14: Issue_Date', 'fy14: Violation_Code',
       'fy14: Vehicle_Body_Type', 'fy14: Vehicle_Make', 'fy14: Issuing_Agency',
       'fy14: Street_Code1', 'fy14: Street_Code2', 'fy14: Street_Code3',
       'fy14: Vehicle_Expiration_Date', 'fy14: Violation_Location',
       'fy14: Violation_Precinct', 'fy14: Issuer_Precinct',
       'fy14: Issuer_Code', 'fy14: Issuer_Command', 'fy14: Issuer_Squad',
       'fy14: Violation_Time', 'fy14: Time_First_Observed',
       'fy14: Violation_County', 'fy14: Violation_In_Front_Of_Or_Opposite',
       'fy14: House_Number', 'fy14: Street_Name', 'fy14: Intersecting_Street',
       'fy14: Date_First_Observed', 'fy14: Law_Section', 'fy14: Sub_Division',
       'fy14: Violation_Legal_Code', 'fy14: Days_Parking_In_Effect_',
       'fy14: From_Hours_In_Effect', 'fy14: To_Hours_In_Effect',
       'fy14: Vehicle_Color', 'fy14: Unregistered_Vehicle?',
       

In [10]:
## Reduce to only matching columns
from functools import reduce

columns = [set(fy14.columns),
           set(fy15.columns),
           set(fy16.columns),
           set(fy17.columns)]
print(columns)
common_columns = list(reduce(lambda x, i: x.intersection(i), columns))
print(common_columns)

[{'Street_Name', 'Plate_ID', 'Vehicle_Expiration_Date', 'Census_Tract', 'Issuer_Squad', 'Issuing_Agency', 'Hydrant_Violation', 'No_Standing_or_Stopping_Violation', 'Issuer_Precinct', 'Violation_Legal_Code', 'Violation_Description', 'Community_Board', 'Violation_County', 'Feet_From_Curb', 'Days_Parking_In_Effect_', 'Vehicle_Make', 'Violation_Precinct', 'Vehicle_Color', 'Plate_Type', 'House_Number', 'Law_Section', 'Vehicle_Body_Type', 'Double_Parking_Violation', 'BBL', 'Violation_Time', 'To_Hours_In_Effect', 'Street_Code1', 'NTA', 'Issuer_Command', 'Violation_In_Front_Of_Or_Opposite', 'Meter_Number', 'Unregistered_Vehicle?', 'Street_Code2', 'Latitude', 'Issuer_Code', 'From_Hours_In_Effect', 'BIN', 'Issue_Date', 'Violation_Location', 'Sub_Division', 'Violation_Post_Code', 'Street_Code3', 'Vehicle_Year', 'Violation_Code', 'Intersecting_Street', 'Date_First_Observed', 'Community_Council_', 'Longitude', 'Summons_Number', 'Registration_State', 'Time_First_Observed'}, {'Street_Name', 'Plate_ID

In [11]:
fy17[common_columns].head()

,Street_Name,Plate_ID,Vehicle_Expiration_Date,Issuer_Squad,Issuing_Agency,Hydrant_Violation,No_Standing_or_Stopping_Violation,Issuer_Precinct,Violation_Legal_Code,Violation_Description,...,Sub_Division,Violation_Post_Code,Street_Code3,Vehicle_Year,Violation_Code,Intersecting_Street,Date_First_Observed,Summons_Number,Registration_State,Time_First_Observed
0,ALLERTON AVE (W/B) @,GZH7067,0,None,V,None,None,0,T,FAILURE TO STOP AT RED LIGHT,...,D,None,0,2001,7,BARNES AVE,0,5092469481,NY,None
1,ALLERTON AVE (W/B) @,GZH7067,0,None,V,None,None,0,T,FAILURE TO STOP AT RED LIGHT,...,D,None,0,2001,7,BARNES AVE,0,5092451658,NY,None
2,SB WEBSTER AVE @ E 1,FZX9232,0,None,V,None,None,0,T,BUS LANE VIOLATION,...,C,None,0,2004,5,94TH ST,0,4006265037,NY,None
3,7th Ave,66623ME,20180630,J,T,None,None,14,None,47-Double PKG-Midtown,...,l2,04,34350,2007,47,None,0,8478629828,NY,None
4,6th Ave,37033JV,20170228,M,T,None,None,13,None,69-Failure to Disp Muni Recpt,...,h1,31 6,34330,2007,69,None,0,7868300310,NY,None


In [12]:
fy14[common_columns].head()

,Street_Name,Plate_ID,Vehicle_Expiration_Date,Issuer_Squad,Issuing_Agency,Hydrant_Violation,No_Standing_or_Stopping_Violation,Issuer_Precinct,Violation_Legal_Code,Violation_Description,...,Sub_Division,Violation_Post_Code,Street_Code3,Vehicle_Year,Violation_Code,Intersecting_Street,Date_First_Observed,Summons_Number,Registration_State,Time_First_Observed
0,W 175 ST,GBB9093,20140831,0000,P,None,None,33,None,None,...,F1,None,21190,2013,46,None,0,1283294138,NY,None
1,W 177 ST,62416MB,20140430,0000,P,None,None,33,None,None,...,C,None,40404,2012,46,None,0,1283294151,NY,None
2,W 163 ST,78755JZ,20140228,0000,P,None,None,33,None,None,...,F7,None,13610,0,46,None,0,1283294163,NY,None
3,W 176 ST,63009MA,20141031,0000,P,None,None,33,None,None,...,F1,None,12010,2010,46,None,0,1283294175,NY,None
4,W 174 ST,91648MC,0,0000,P,None,None,33,None,None,...,E1,None,31190,2012,41,None,0,1283294187,NY,None


In [13]:
## verify that dtypes are the same
import numpy as np
import pandas as pd

dtype_tuples = [(x, np.str) for x in common_columns]
dtypes = dict(dtype_tuples)
print(dtypes)

fy14 = dd.read_parquet('s3://dask/2014_parking_violations/', dtype=dtypes)

with ProgressBar():
    display(fy14[common_columns].head())

{'Street_Name': <class 'str'>, 'Plate_ID': <class 'str'>, 'Vehicle_Expiration_Date': <class 'str'>, 'Issuer_Squad': <class 'str'>, 'Issuing_Agency': <class 'str'>, 'Hydrant_Violation': <class 'str'>, 'No_Standing_or_Stopping_Violation': <class 'str'>, 'Issuer_Precinct': <class 'str'>, 'Violation_Legal_Code': <class 'str'>, 'Violation_Description': <class 'str'>, 'Violation_County': <class 'str'>, 'Feet_From_Curb': <class 'str'>, 'Days_Parking_In_Effect_': <class 'str'>, 'Vehicle_Make': <class 'str'>, 'Violation_Precinct': <class 'str'>, 'Vehicle_Color': <class 'str'>, 'Plate_Type': <class 'str'>, 'House_Number': <class 'str'>, 'Law_Section': <class 'str'>, 'Vehicle_Body_Type': <class 'str'>, 'Double_Parking_Violation': <class 'str'>, 'Violation_Time': <class 'str'>, 'To_Hours_In_Effect': <class 'str'>, 'Street_Code1': <class 'str'>, 'Issuer_Command': <class 'str'>, 'Violation_In_Front_Of_Or_Opposite': <class 'str'>, 'Meter_Number': <class 'str'>, 'Unregistered_Vehicle?': <class 'str'>,

,Street_Name,Plate_ID,Vehicle_Expiration_Date,Issuer_Squad,Issuing_Agency,Hydrant_Violation,No_Standing_or_Stopping_Violation,Issuer_Precinct,Violation_Legal_Code,Violation_Description,...,Sub_Division,Violation_Post_Code,Street_Code3,Vehicle_Year,Violation_Code,Intersecting_Street,Date_First_Observed,Summons_Number,Registration_State,Time_First_Observed
0,W 175 ST,GBB9093,20140831,0000,P,None,None,33,None,None,...,F1,None,21190,2013,46,None,0,1283294138,NY,None
1,W 177 ST,62416MB,20140430,0000,P,None,None,33,None,None,...,C,None,40404,2012,46,None,0,1283294151,NY,None
2,W 163 ST,78755JZ,20140228,0000,P,None,None,33,None,None,...,F7,None,13610,0,46,None,0,1283294163,NY,None
3,W 176 ST,63009MA,20141031,0000,P,None,None,33,None,None,...,F1,None,12010,2010,46,None,0,1283294175,NY,None
4,W 174 ST,91648MC,0,0000,P,None,None,33,None,None,...,E1,None,31190,2012,41,None,0,1283294187,NY,None


In [15]:
with ProgressBar():
    print(fy14['Vehicle_Year'].unique().head(10))

[########################################] | 100% Completed |  1.5s
0    2013
1    2012
2       0
3    2010
4    2011
5    2001
6    2005
7    1998
8    1995
9    2003
Name: Vehicle_Year, dtype: object


In [16]:
with ProgressBar():
    print(fy14['Vehicle_Year'].isnull().values.any().compute())

[########################################] | 100% Completed | 32.5s
False


In [17]:
dtypes

{'Street_Name': str,
 'Plate_ID': str,
 'Vehicle_Expiration_Date': str,
 'Issuer_Squad': str,
 'Issuing_Agency': str,
 'Hydrant_Violation': str,
 'No_Standing_or_Stopping_Violation': str,
 'Issuer_Precinct': str,
 'Violation_Legal_Code': str,
 'Violation_Description': str,
 'Violation_County': str,
 'Feet_From_Curb': str,
 'Days_Parking_In_Effect_': str,
 'Vehicle_Make': str,
 'Violation_Precinct': str,
 'Vehicle_Color': str,
 'Plate_Type': str,
 'House_Number': str,
 'Law_Section': str,
 'Vehicle_Body_Type': str,
 'Double_Parking_Violation': str,
 'Violation_Time': str,
 'To_Hours_In_Effect': str,
 'Street_Code1': str,
 'Issuer_Command': str,
 'Violation_In_Front_Of_Or_Opposite': str,
 'Meter_Number': str,
 'Unregistered_Vehicle?': str,
 'Street_Code2': str,
 'Issuer_Code': str,
 'From_Hours_In_Effect': str,
 'Issue_Date': str,
 'Violation_Location': str,
 'Sub_Division': str,
 'Violation_Post_Code': str,
 'Street_Code3': str,
 'Vehicle_Year': str,
 'Violation_Code': str,
 'Intersecti

In [25]:
#fy14 = dd.read_parquet('s3://dask/2014_parking_violations/', dtype=dtypes, usecols=common_columns)
#fy15 = dd.read_parquet('s3://dask/2015_parking_violations/', dtype=dtypes, usecols=common_columns)
#fy16 = dd.read_parquet('s3://dask/2016_parking_violations/', dtype=dtypes, usecols=common_columns)
#fy17 = dd.read_parquet('s3://dask/2017_parking_violations/', dtype=dtypes, usecols=common_columns)
data = [fy14, fy15]
nyc_raw = dd.concat(data)

In [26]:
nyc_raw.head()

,Summons_Number,Plate_ID,Registration_State,Plate_Type,Issue_Date,Violation_Code,Vehicle_Body_Type,Vehicle_Make,Issuing_Agency,Street_Code1,...,Hydrant_Violation,Double_Parking_Violation,Latitude,Longitude,Community_Board,Community_Council_,Census_Tract,BIN,BBL,NTA
0,1283294138,GBB9093,NY,PAS,08/04/2013,46,SUBN,AUDI,P,37250,...,None,None,None,None,None,None,None,None,None,None
1,1283294151,62416MB,NY,COM,08/04/2013,46,VAN,FORD,P,37290,...,None,None,None,None,None,None,None,None,None,None
2,1283294163,78755JZ,NY,COM,08/05/2013,46,P-U,CHEVR,P,37030,...,None,None,None,None,None,None,None,None,None,None
3,1283294175,63009MA,NY,COM,08/05/2013,46,VAN,FORD,P,37270,...,None,None,None,None,None,None,None,None,None,None
4,1283294187,91648MC,NY,COM,08/08/2013,41,TRLR,GMC,P,37240,...,None,None,None,None,None,None,None,None,None,None
